In [111]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [112]:
# hyperparameter
batch_size = 64
block_size = 256 # context length
max_iters = 10000
eval_iters = 1000
eval_interval = 300
n_embd = 384
n_head = 6
n_layer = 6   # no of blocks
dropout = 0.2

learning_rate = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(1337)


In [113]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-07-29 17:18:14--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.9’

input.txt.9         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-07-29 17:18:14 (16.7 MB/s) - ‘input.txt.9’ saved [1115394/1115394]



In [114]:
# read dataset
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
# Mapping sets
stoi = {ch:i for i ,ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
# encode str -> int
encode = lambda s:[stoi[c] for c in s]
# decode int -> str
decode = lambda l:''.join([itos[i] for i in l])

In [115]:
data = torch.tensor(encode(text), dtype = torch.long)
# split into train and val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,)) # predicting the 1st encoded index
  # print(f"the ix is :{ix}")
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
      print(f"{k} in loss:{loss.item()}")
    out[split] = losses.mean()
  model.train()
  return out


In [116]:
class Head(nn.Module):
  "single head self attention"
  def __init__(self, head_size) -> None:
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias= False)
    self.query = nn.Linear(n_embd, head_size, bias= False)
    self.value = nn.Linear(n_embd, head_size, bias= False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x) # (B, T, C)
    q = self.query(x) # (B, T, C)
    # Attention scores
    wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B , C, T) -> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim= -1)
    wei = self.dropout(wei)
    # Weight aggregation
    v = self.value(x)
    out = wei @ v
    return out

class MultiHeadAttention(nn.Module):
  " Multi head attention"
  def __init__(self, num_head, head_size) -> None:
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim = -1)
    out = self.dropout(self.proj(out))
    return out


In [117]:
class FeedForward(nn.Module):
  def __init__(self, n_embd) -> None:
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4*n_embd),
        nn.ReLU(),
        nn.Linear(4*n_embd, n_embd), # projection going back into resuidal path way
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

In [118]:
class Block(nn.Module):
  def __init__(self, n_embd, n_head) -> None:
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x)) # adding resudial connection
    x = x + self.ffwd(self.ln1(x))
    return x

In [119]:
class BiagramModel(nn.Module):

  def __init__(self) -> None:
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks= nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
    self.ln_f  = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, target= None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx) # (B, T, C)
    pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (B ,T)
    x = tok_emb + pos_emb # (B, T, C)
    x = self.blocks(x)
    x= self.ln_f(x)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)

      loss = F.cross_entropy(logits, target) # special case for multiple dimensions

    return logits , loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      # Crop to prevent overflow
      idx_count = idx[:, -block_size:]
      # getting the predictions
      logits, loss = self(idx_count)
      # focus on last time step
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim = -1) # (B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim = 1)# (B, T+1)
    return idx

In [120]:
model = BiagramModel()
m = model.to(device)
# Model optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [121]:
# Training loop
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step{iter}: train loss {losses['train']:.4f}, val_loss {losses['val']:.4f}")

  xb, yb = get_batch('train')

  # Evaluate loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none= True)
  loss.backward()
  optimizer.step()

0 in loss:4.2896199226379395
1 in loss:4.286368370056152
2 in loss:4.275571823120117
3 in loss:4.286287307739258
4 in loss:4.290957450866699
5 in loss:4.287097454071045
6 in loss:4.287629127502441
7 in loss:4.281123638153076
8 in loss:4.288971900939941
9 in loss:4.284603118896484
10 in loss:4.281246185302734
11 in loss:4.288468837738037
12 in loss:4.279931545257568
13 in loss:4.28678560256958
14 in loss:4.281149387359619
15 in loss:4.288675308227539
16 in loss:4.280574798583984
17 in loss:4.284282207489014
18 in loss:4.2895097732543945


KeyboardInterrupt: 

In [ ]:
# generate from model
context = torch.zeros((1,1), dtype= torch.long, device = device)
print(decode(m.generate(context, max_new_tokens= 100)[0].tolist()))